In [1]:
from urllib.request import urlopen
import certifi
import json
from datetime import datetime

import pandas as pd

In [2]:
API_KEY="e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c"

FETCH_STOCKS_URL = f"https://financialmodelingprep.com/api/v3/stock-screener?apikey={API_KEY}&isActivelyTrading=true&exchange=nyse,nasdaq&limit=10000&marketCapMoreThan=10000&volumeMoreThan=10000"
# gross profit, earningspersharebasic
FETCH_INCOME_URL = f"https://financialmodelingprep.com/api/v3/income-statement/%s?period=quarter&limit=5&apikey={API_KEY}"
# totalAssets
FETCH_BALANCE_URL = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/%s?period=quarter&limit=5&apikey={API_KEY}"
# priceToSalesRatio
FETCH_RATIO_URL = f"https://financialmodelingprep.com/api/v3/key-metrics/%s?period=quarter&limit=5&apikey={API_KEY}"
# grossProfitGrowth, ebitgrowth
FETCH_GROWTH_URL = f"https://financialmodelingprep.com/api/v3/financial-growth/%s?period=quarter&limit=5&apikey={API_KEY}"

In [3]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [4]:
stocks = get_jsonparsed_data(FETCH_STOCKS_URL)

/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_84101/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


In [5]:
tickers = list(map(lambda stock: stock["symbol"], stocks))
tickers = sorted(tickers)

In [80]:
# 0 < psr < 2
# 0 < gross profit / total asset < 2
# 0 < gross_profit
# 0 < eps
# 0.1 < ebit growth
# 0.1 < gross profit growth

In [6]:
def fetch_metric(key, data):
    return data[key] if key in data else 0

def build_key_metrics(date, income, balance, ratio, growth):
    return {
        "date" : date,
        "year" : date.year,
        "month" : date.month,
        "period" : income["period"],
        "symbol" : income["symbol"],
        "gross_profit" : fetch_metric("grossProfit", income),
        "eps" : fetch_metric("eps", income),
        "assets" : fetch_metric("totalAssets", balance),
        "gp_a" : fetch_metric("grossProfit", income) / fetch_metric("totalAssets", balance),
        "psr" : fetch_metric("priceToSalesRatio", ratio),
        "gross_profit_growth" : fetch_metric("grossProfitGrowth", growth),
        "ebit_growth" : fetch_metric("ebitgrowth", growth)
    }

def fetch_metrics(ticker):
    print(f"fetch {ticker}")
    incomes = get_jsonparsed_data(FETCH_INCOME_URL % ticker)
    balances = get_jsonparsed_data(FETCH_BALANCE_URL % ticker)
    ratios = get_jsonparsed_data(FETCH_RATIO_URL % ticker)
    growths = get_jsonparsed_data(FETCH_GROWTH_URL % ticker)
    
    results = []
    
    size = min(len(incomes), len(balances), len(ratios), len(growths))
    
    for i in range(size):
        income = incomes[i]
        balance = balances[i]
        ratio = ratios[i]
        growth = growths[i]
    
        date = datetime.strptime(income["fillingDate"], "%Y-%m-%d")
        
        results.append(build_key_metrics(date, income, balance, ratio, growth))
    
    return results

def corps_total_rank_by_standard_columns(df, column_map) -> pd.DataFrame:
    report_df = df.copy()
    rank_df = pd.DataFrame(index=report_df.index)
    total_value = pd.Series([0] * len(report_df), index=report_df.index)

    for col, ascending in column_map.items():
        total_value += report_df[col].rank(ascending=ascending)

    rank_df['total_rank'] = total_value.rank()
    rank_df = rank_df.sort_values(by='total_rank')
    report_df['rank'] = rank_df['total_rank']

    return report_df.loc[rank_df.index]

In [60]:
metrics = []

for ticker in tickers:
    try:
        metrics.append(fetch_metrics(ticker))
    except:
        None

fetch A


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_30685/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


fetch AA
fetch AACG
fetch AACI
fetch AACT
fetch AADI
fetch AAL
fetch AAN
fetch AAOI
fetch AAON
fetch AAP
fetch AAPB
fetch AAPD
fetch AAPL
fetch AAPU
fetch AAT
fetch AAXJ
fetch AB
fetch ABAT
fetch ABBV
fetch ABCB
fetch ABCL
fetch ABEO
fetch ABEV
fetch ABG
fetch ABIO
fetch ABL
fetch ABLV
fetch ABM
fetch ABNB
fetch ABOS
fetch ABR
fetch ABSI
fetch ABT
fetch ABUS
fetch ABVC
fetch ABVX
fetch ACA
fetch ACAC
fetch ACAD
fetch ACB
fetch ACCD
fetch ACCO
fetch ACDC
fetch ACEL
fetch ACET
fetch ACGL
fetch ACHC
fetch ACHL
fetch ACHR
fetch ACHV
fetch ACI
fetch ACIC
fetch ACIU
fetch ACIW
fetch ACLS
fetch ACLX
fetch ACM
fetch ACMR
fetch ACN
fetch ACON
fetch ACOR
fetch ACP
fetch ACR
fetch ACRE
fetch ACRS
fetch ACRV
fetch ACST
fetch ACT
fetch ACTG
fetch ACV
fetch ACVA
fetch ACWI
fetch ACWX
fetch ACXP
fetch ADAG
fetch ADAP
fetch ADBE
fetch ADC
fetch ADCT
fetch ADD
fetch ADEA
fetch ADI
fetch ADIL
fetch ADM
fetch ADMA
fetch ADN
fetch ADNT
fetch ADP
fetch ADPT
fetch ADSE
fetch ADSK
fetch ADT
fetch ADTH
fetch 

fetch BRKL
fetch BRKR
fetch BRLT
fetch BRNS
fetch BRO
fetch BROG
fetch BROS
fetch BRP
fetch BRRR
fetch BRSH
fetch BRSP
fetch BRT
fetch BRTX
fetch BRW
fetch BRX
fetch BRY
fetch BRZE
fetch BSAC
fetch BSBR
fetch BSCO
fetch BSCP
fetch BSCQ
fetch BSCR
fetch BSCS
fetch BSCT
fetch BSCU
fetch BSCV
fetch BSCW
fetch BSCX
fetch BSET
fetch BSFC
fetch BSGM
fetch BSIG
fetch BSJO
fetch BSJP
fetch BSJQ
fetch BSJR
fetch BSJT
fetch BSJU
fetch BSL
fetch BSM
fetch BSMO
fetch BSMP
fetch BSMQ
fetch BSMR
fetch BSMS
fetch BSMT
fetch BSMU
fetch BSRR
fetch BST
fetch BSTZ
fetch BSVO
fetch BSX
fetch BSY
fetch BTA
fetch BTAI
fetch BTBT
fetch BTCM
fetch BTCS
fetch BTCT
fetch BTCTW
fetch BTCY
fetch BTDR
fetch BTE
fetch BTF
fetch BTFX
fetch BTI
fetch BTM
fetch BTO
fetch BTOG
fetch BTSG
fetch BTT
fetch BTTX
fetch BTU
fetch BTZ
fetch BUD
fetch BUG
fetch BUI
fetch BUJA
fetch BUR
fetch BURL
fetch BURU
fetch BUSE
fetch BUXX
fetch BV
fetch BVN
fetch BVS
fetch BW
fetch BWA
fetch BWAY
fetch BWB
fetch BWEN
fetch BWG
fetch BWM

fetch DUK
fetch DUO
fetch DUOL
fetch DUOT
fetch DV
fetch DVA
fetch DVAL
fetch DVAX
fetch DVN
fetch DVY
fetch DWAS
fetch DWAT
fetch DWSH
fetch DWSN
fetch DX
fetch DXC
fetch DXCM
fetch DXJS
fetch DXLG
fetch DXPE
fetch DXYN
fetch DXYZ
fetch DY
fetch DYAI
fetch DYCQU
fetch DYN
fetch DYNT
fetch DYTA
fetch DZSI
fetch E
fetch EA
fetch EAD
fetch EAF
fetch EARN
fetch EAT
fetch EB
fetch EBAY
fetch EBC
fetch EBET
fetch EBF
fetch EBMT
fetch EBON
fetch EBS
fetch EBTC
fetch EC
fetch ECAT
fetch ECC
fetch ECF
fetch ECL
fetch ECO
fetch ECOW
fetch ECPG
fetch ECVT
fetch ECX
fetch ED
fetch EDAP
fetch EDBL
fetch EDD
fetch EDF
fetch EDIT
fetch EDN
fetch EDR
fetch EDSA
fetch EDTK
fetch EDU
fetch EDUC
fetch EE
fetch EEFT
fetch EEMA
fetch EETH
fetch EEX
fetch EFC
fetch EFR
fetch EFSC
fetch EFT
fetch EFTR
fetch EFX
fetch EFXT
fetch EG
fetch EGAN
fetch EGBN
fetch EGHT
fetch EGIO
fetch EGLE
fetch EGO
fetch EGOX
fetch EGP
fetch EGRX
fetch EGY
fetch EH
fetch EHAB
fetch EHC
fetch EHI
fetch EHTH
fetch EIC
fetch EIG
f

fetch HCP
fetch HCSG
fetch HCTI
fetch HCVI
fetch HD
fetch HDB
fetch HDSN
fetch HE
fetch HEAR
fetch HEES
fetch HEI
fetch HELE
fetch HEPA
fetch HEPS
fetch HEQ
fetch HERD
fetch HERO
fetch HES
fetch HESM
fetch HFFG
fetch HFRO
fetch HFWA
fetch HG
fetch HGBL
fetch HGLB
fetch HGTY
fetch HGV
fetch HHGC
fetch HHS
fetch HI
fetch HIBB
fetch HIE
fetch HIG
fetch HII
fetch HIMS
fetch HIMX
fetch HIO
fetch HIPO
fetch HITI
fetch HIVE
fetch HIW
fetch HIX
fetch HKD
fetch HKIT
fetch HL
fetch HLAL
fetch HLF
fetch HLI
fetch HLIO
fetch HLIT
fetch HLLY
fetch HLMN
fetch HLN
fetch HLNE
fetch HLP
fetch HLT
fetch HLTH
fetch HLVX
fetch HLX
fetch HLXB
fetch HMC
fetch HMN
fetch HMST
fetch HMY
fetch HNDL
fetch HNI
fetch HNRG
fetch HNST
fetch HNW
fetch HOFT
fetch HOG
fetch HOLI
fetch HOLO
fetch HOLX
fetch HOMB
fetch HON
fetch HONE
fetch HOOD
fetch HOOK
fetch HOPE
fetch HOTH
fetch HOUR
fetch HOUS
fetch HOV
fetch HOVR
fetch HOWL
fetch HP
fetch HPCO
fetch HPE
fetch HPF
fetch HPI
fetch HPK
fetch HPP
fetch HPQ
fetch HPS
fe

fetch MAIN
fetch MAMA
fetch MAMO
fetch MAN
fetch MANH
fetch MANU
fetch MAPS
fetch MAR
fetch MARA
fetch MARK
fetch MARPS
fetch MAS
fetch MASI
fetch MASS
fetch MAT
fetch MATH
fetch MATV
fetch MATW
fetch MATX
fetch MAV
fetch MAX
fetch MAXI
fetch MAXN
fetch MBB
fetch MBC
fetch MBCN
fetch MBI
fetch MBIN
fetch MBIO
fetch MBLY
fetch MBOT
fetch MBRX
fetch MBTC
fetch MBUU
fetch MBWM
fetch MC
fetch MCB
fetch MCBC
fetch MCBS
fetch MCD
fetch MCFT
fetch MCHI
fetch MCHP
fetch MCHX
fetch MCI
fetch MCK
fetch MCN
fetch MCO
fetch MCOM
fetch MCR
fetch MCRB
fetch MCRI
fetch MCS
fetch MCW
fetch MCY
fetch MD
fetch MDAI
fetch MDB
fetch MDC
fetch MDGL
fetch MDGS
fetch MDIA
fetch MDIV
fetch MDLZ
fetch MDRX
fetch MDT
fetch MDU
fetch MDV
fetch MDWD
fetch MDXG
fetch MDXH
fetch ME
fetch MEC
fetch MED
fetch MEDP
fetch MEDS
fetch MEG
fetch MEGI
fetch MEGL
fetch MEI
fetch MEIP
fetch MELI
fetch MEOH
fetch MERC
fetch MESA
fetch MESO
fetch MET
fetch META
fetch METC
fetch METCB
fetch MFA
fetch MFC
fetch MFD
fetch MFG
fet

fetch PDT
fetch PEB
fetch PEBO
fetch PECO
fetch PEG
fetch PEGA
fetch PEGR
fetch PEGY
fetch PEN
fetch PENN
fetch PEO
fetch PEP
fetch PEPG
fetch PERF
fetch PERI
fetch PESI
fetch PET
fetch PETQ
fetch PETS
fetch PETZ
fetch PEV
fetch PEY
fetch PFBC
fetch PFC
fetch PFD
fetch PFE
fetch PFF
fetch PFG
fetch PFGC
fetch PFIE
fetch PFIS
fetch PFL
fetch PFLT
fetch PFM
fetch PFMT
fetch PFN
fetch PFO
fetch PFS
fetch PFSI
fetch PG
fetch PGC
fetch PGEN
fetch PGJ
fetch PGNY
fetch PGP
fetch PGR
fetch PGRE
fetch PGRU
fetch PGSS
fetch PGY
fetch PGZ
fetch PH
fetch PHAT
fetch PHD
fetch PHG
fetch PHI
fetch PHIN
fetch PHIO
fetch PHK
fetch PHM
fetch PHO
fetch PHR
fetch PHT
fetch PHUN
fetch PHVS
fetch PHX
fetch PHYS
fetch PI
fetch PID
fetch PIE
fetch PII
fetch PIII
fetch PIK
fetch PIM
fetch PINC
fetch PINE
fetch PINS
fetch PIPR
fetch PIRS
fetch PIXY
fetch PJT
fetch PK
fetch PKBK
fetch PKE
fetch PKG
fetch PKOH
fetch PKST
fetch PKW
fetch PKX
fetch PL
fetch PLAB
fetch PLAY
fetch PLBY
fetch PLCE
fetch PLD
fetch PLL


fetch SONY
fetch SOPA
fetch SOPH
fetch SOR
fetch SOS
fetch SOTK
fetch SOUN
fetch SOXQ
fetch SOXX
fetch SP
fetch SPB
fetch SPBC
fetch SPC
fetch SPCB
fetch SPCE
fetch SPE
fetch SPEC
fetch SPG
fetch SPGC
fetch SPGI
fetch SPH
fetch SPHR
fetch SPI
fetch SPIR
fetch SPKL
fetch SPNS
fetch SPNT
fetch SPOK
fetch SPOT
fetch SPPL
fetch SPR
fetch SPRB
fetch SPRC
fetch SPRO
fetch SPRU
fetch SPRX
fetch SPRY
fetch SPSC
fetch SPT
fetch SPTN
fetch SPWH
fetch SPWR
fetch SPXC
fetch SPXX
fetch SQ
fetch SQFT
fetch SQM
fetch SQNS
fetch SQQQ
fetch SQSP
fetch SR
fetch SRAD
fetch SRCE
fetch SRCL
fetch SRDX
fetch SRE
fetch SRET
fetch SRFM
fetch SRG
fetch SRI
fetch SRM
fetch SRPT
fetch SRRK
fetch SRTS
fetch SRV
fetch SRZN
fetch SSB
fetch SSBK
fetch SSD
fetch SSKN
fetch SSL
fetch SSNC
fetch SSNT
fetch SSP
fetch SSRM
fetch SSSS
fetch SST
fetch SSTI
fetch SSTK
fetch SSYS
fetch ST
fetch STAA
fetch STAF
fetch STAG
fetch STBA
fetch STBX
fetch STC
fetch STCN
fetch STE
fetch STEL
fetch STEM
fetch STEP
fetch STER
fetch ST

fetch WMG
fetch WMK
fetch WMS
fetch WMT
fetch WNC
fetch WNEB
fetch WNS
fetch WNW
fetch WOLF
fetch WOOF
fetch WOR
fetch WORX
fetch WOW
fetch WPC
fetch WPM
fetch WPP
fetch WPRT
fetch WRAP
fetch WRB
fetch WRBY
fetch WRK
fetch WRLD
fetch WS
fetch WSBC
fetch WSBF
fetch WSC
fetch WSFS
fetch WSM
fetch WSO
fetch WSR
fetch WST
fetch WT
fetch WTBA
fetch WTFC
fetch WTI
fetch WTMA
fetch WTRG
fetch WTS
fetch WTTR
fetch WTW
fetch WU
fetch WULF
fetch WVE
fetch WW
fetch WWD
fetch WWW
fetch WY
fetch WYNN
fetch X
fetch XAIR
fetch XBIL
fetch XBIT
fetch XCUR
fetch XEL
fetch XELA
fetch XELB
fetch XENE
fetch XERS
fetch XFIN
fetch XFLT
fetch XFOR
fetch XGN
fetch XHR
fetch XLO
fetch XMTR
fetch XNCR
fetch XNET
fetch XOM
fetch XOMA
fetch XOS
fetch XP
fetch XPEL
fetch XPER
fetch XPEV
fetch XPO
fetch XPOF
fetch XPON
fetch XPRO
fetch XRAY
fetch XRTX
fetch XRX
fetch XTIA
fetch XTKG
fetch XTLB
fetch XWEL
fetch XXII
fetch XYL
fetch YALA
fetch YELP
fetch YETI
fetch YEXT
fetch YGMZ
fetch YI
fetch YMAB
fetch YMM
fetch Y

In [28]:
stocks_df = pd.DataFrame(metrics)

In [29]:
stocks_df

,date,year,month,period,symbol,gross_profit,eps,assets,gp_a,psr,gross_profit_growth,ebit_growth
0,2024-03-05,2024,3,Q1,A,913000000.0,1.1900,10948000000,0.083394,22.991134,-0.002186,-0.019608
1,2024-02-21,2024,2,Q4,AA,7000000.0,-0.8400,14158000000,0.000494,2.338291,-0.681818,0.598870
2,2023-11-13,2023,11,Q1,AACG,29224857.0,-0.2400,466256544,0.062680,4.155527,1.042632,0.631506
3,2024-02-16,2024,2,Q1,AACI,-571529.0,-0.0446,25939504,-0.022033,0.000000,0.062874,0.024944
4,2023-12-31,2023,12,Q4,AACT,0.0,0.1100,525453425,0.000000,0.000000,0.000000,-0.123593
...,...,...,...,...,...,...,...,...,...,...,...,...
4998,2024-04-01,2024,4,Q4,ZVRA,10431000.0,-0.4000,172327000,0.060530,18.782223,2.791712,0.045105
4999,2024-03-25,2024,3,Q4,ZVSA,-2600.0,-3.2900,22114284,-0.000118,0.000000,0.000000,0.388075
5000,2023-10-31,2023,10,Q3,ZWS,169900000.0,0.2400,2812900000,0.060400,12.186731,-0.020749,0.096715
5001,2024-03-06,2024,3,Q4,ZYME,16926000.0,0.0000,580880000,0.029139,0.000000,2.029750,0.295334


In [37]:
filtered_stocks = stocks_df.query((
    "0 < psr and psr < 2 and "
    "0 < gp_a and gp_a < 2 and "
    "0 < gross_profit and "
    "0 < eps and "
    "0.1 < ebit_growth and "
    "0.1 < gross_profit_growth"
))
sorted_stocks = corps_total_rank_by_standard_columns(filtered_stocks, {
    'psr': True,
    'gp_a': False
})

In [39]:
sorted_stocks.head(10)

,date,year,month,period,symbol,gross_profit,eps,assets,gp_a,psr,gross_profit_growth,ebit_growth,rank
4371,2023-06-30,2023,6,Q2,TCTM,2.787110e+08,0.70,1226389000,0.227261,0.137931,0.386601,0.884348,1.0
1870,2024-03-27,2024,3,Q4,GCO,3.420670e+08,2.49,1329890000,0.257215,0.425395,0.228579,2.549392,2.0
574,2024-03-15,2024,3,Q4,BFRI,4.933000e+06,1.65,27932000,0.176607,0.559542,0.125228,0.163420,3.0
1751,2024-02-08,2024,2,Q2,FLWS,3.556970e+08,0.97,1191710000,0.298476,0.850213,2.489689,3.428347,4.5
1321,2024-03-22,2024,3,Q4,DLTH,1.097080e+08,0.21,556478000,0.197147,0.666556,0.580716,1.771403,4.5
1659,2024-02-08,2024,2,Q2,FARM,3.610900e+07,0.13,181846000,0.198569,0.709085,0.172827,3.131705,7.0
4121,2024-02-08,2024,2,Q2,SLQT,2.138340e+08,0.12,1235919000,0.173016,0.568862,0.334636,3.466830,7.0
2472,2024-03-19,2024,3,Q4,JWN,1.605000e+09,0.82,8444000000,0.190076,0.666176,0.294355,1.217199,7.0
100,2023-12-31,2023,12,Q2,AENT,4.770300e+07,0.18,428015000,0.111452,0.111080,0.745700,4.949495,9.5
4216,2024-03-13,2024,3,Q4,SPRO,7.178900e+07,0.97,182390000,0.393602,1.084679,1.830239,26.251293,9.5


In [62]:
result = []

for item in metrics:
    result.extend(item)

In [65]:
stocks_df = pd.DataFrame(result)

In [88]:
filtered_stocks = stocks_df[
    (stocks_df['month'].isin([1, 2, 3, 4])) & (stocks_df['year'] == 2024) |
    (stocks_df['month'].isin([12])) & (stocks_df['year'] == 2023)
].query((
    "0 < psr and psr < 2 and "
    "0 < gp_a and gp_a < 2 and "
    "0 < gross_profit and "
    "0 < eps and "
    "0.1 < ebit_growth and "
    "0.1 < gross_profit_growth"
))
sorted_stocks = corps_total_rank_by_standard_columns(filtered_stocks, {
    'psr': True,
    'gp_a': False
})

In [89]:
sorted_stocks.head(10)

,date,year,month,period,symbol,gross_profit,eps,assets,gp_a,psr,gross_profit_growth,ebit_growth,rank
9219,2024-03-27,2024,3,Q4,GCO,3.420670e+08,2.49,1.329890e+09,0.257215,0.425395,0.228579,2.549392,1.0
9220,2023-12-07,2023,12,Q3,GCO,2.784250e+08,0.60,1.492931e+09,0.186496,0.487416,0.115842,1.280943,2.0
2828,2024-03-15,2024,3,Q4,BFRI,4.933000e+06,1.65,2.793200e+07,0.176607,0.559542,0.125228,0.163420,3.5
6504,2024-03-22,2024,3,Q4,DLTH,1.097080e+08,0.21,5.564780e+08,0.197147,0.666556,0.580716,1.771403,3.5
8173,2024-02-08,2024,2,Q2,FARM,3.610900e+07,0.13,1.818460e+08,0.198569,0.709085,0.172827,3.131705,6.0
12172,2024-03-19,2024,3,Q4,JWN,1.605000e+09,0.82,8.444000e+09,0.190076,0.666176,0.294355,1.217199,6.0
8628,2024-02-08,2024,2,Q2,FLWS,3.556970e+08,0.97,1.191710e+09,0.298476,0.850213,2.489689,3.428347,6.0
20305,2024-02-08,2024,2,Q2,SLQT,2.138340e+08,0.12,1.235919e+09,0.173016,0.568862,0.334636,3.466830,8.0
3298,2023-12-07,2023,12,Q2,BNED,1.362420e+08,0.46,1.144979e+09,0.118991,0.083937,1.690722,1.858704,9.0
20777,2024-03-13,2024,3,Q4,SPRO,7.178900e+07,0.97,1.823900e+08,0.393602,1.084679,1.830239,26.251293,10.0
